# Clean download

In [18]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup
from newspaper import Article
from selenium import webdriver
import nltk


In [4]:
pages_to_scrape_world = [f"https://www.tvp.info/191867/swiat?page={i}" for i in range(1,300)]
pages_to_scrape_poland = [f"https://www.tvp.info/191866/polska?page={i}" for i in range(1,500)]
pages_to_scrape = pages_to_scrape_world + pages_to_scrape_poland
domain = "https://www.tvp.info"

In [5]:
start = time.process_time()
titles_all = []
urls_all = []
locations_all = []
pages_all = []
dates_all = []
descriptions_all = []
for page in pages_to_scrape:
    # open the page with Selenium to get the js to render (requests don't work)
    browser = webdriver.Chrome(executable_path=r"C:\Users\Jedrek\Documents\scraping_NLP\chromedriver.exe")
    browser.get(page)
    source_code = (browser.page_source)
    browser.quit()
    # soup it up
    coverpage = source_code
    soup1 = BeautifulSoup(coverpage, 'html.parser')
    regex = re.compile('.*main-mesh-box__text-container*')
    coverpage_news = soup1.find_all("div", {"class" : regex})
    dates_news = soup1.find_all("span", {"class" : 'time-article__time'})
    desc_news = soup1.find_all("div", {"class" : 'main-mesh-box__description'})
    location_news = soup1.find_all("span", {"class" : 'time-article__category'})
    locations = [i.get_text() for i in location_news]
    dates = [i.get_text() for i in dates_news]
    descriptions = [i.get_text().strip() for i in desc_news]
    # urls and titles 
    regex_h = re.compile('[h1|h2|h3]')
    titles = []
    for i in range(len(coverpage_news)):
        try:
            t = coverpage_news[i].find(regex_h).get_text() 
            titles.append(t.strip())
        except:
            titles.append("NA")
    pages = [page[-1] for i in titles] 
    urls = [f"{domain}{coverpage_news[i].find('a')['href']}" for i in range(len(coverpage_news))]
    titles_all = titles_all + titles
    urls_all = urls_all + urls
    locations_all = locations_all + locations
    pages_all = pages_all + pages
    dates_all = dates_all + dates
    descriptions_all = descriptions_all + descriptions

pattern = "(?i)(epidem\w*)|(covid\w*\W*1?9?)|(pandem\w*)|(koronaw*)|(zakaż\w*\W*)"
column_names = ['Location', 'Page_n', 'Title', 'Url', 'Date', 'Description']
df = pd.DataFrame(list(zip(locations_all, pages_all, titles_all, urls_all, dates_all, descriptions_all)), columns = column_names)
df_corona = df[df["Title"].str.contains(pattern)]
df.to_excel("df_full_20210523.xlsx")
clean_texts = []
for url in df_corona['Url']:
    try:
        a = Article(url, language='pl')
        a.download()
        a.parse()
        list_clean = [i for i in (a.text).split("\n\n") if  "wieszwiecej" not in i and "zobacz więcej" not in i and "źródło" not in i and "KORONAWIRUS – RAPORT" not in i]
        text_clean = ' '.join(list_clean).strip()
        clean_texts.append(text_clean)
    except:
        clean_texts.append("ERROR")

df_corona['Clean_text'] = pd.Series(clean_texts).values
df_corona.to_excel("df_corona_20210523.xlsx")


elapsed_time = time.process_time() - start
elapsed_time

C:\Users\Jedrek\anaconda3\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-5-9acb98f95a3f>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corona['Clean_text'] = pd.Series(clean_texts).values


72.0625